[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/prashantkul/simple-mcp-server/blob/claude/mcp-colab-customer-demo-011CUyKnnHZc4KxRNuBxQry3/adk_customer_agent.ipynb)

# 🤖 Google ADK Customer Management Agent

## Using MCPToolSet to Connect to Our MCP Server

This notebook demonstrates how to build an AI agent using **Google Agent Development Kit (ADK)** that connects to our customer management MCP server.

**What you'll learn:**
- How to install and configure Google ADK
- How to use MCPToolSet to connect to MCP servers
- How to create an agent that uses MCP tools
- How to interact with the agent using natural language
- Testing the agent with various customer management scenarios
- Deploying the agent with ADK Web and ngrok for public access

**Prerequisites:**
1. Complete the `mcp_customer_demo.ipynb` notebook first
2. Have your MCP server running with a public ngrok URL
3. Have a Google API key for Gemini
4. Have ngrok auth token in Colab secrets (same as MCP server notebook)

Let's get started! 🚀

---

## 📚 What is Google ADK?

**Google Agent Development Kit (ADK)** is a framework for building AI agents powered by Google's Gemini models.

### Key Features:

- **LLM Agents**: Create intelligent agents powered by Gemini
- **Tool Integration**: Connect agents to external tools and APIs
- **MCP Support**: Built-in support for Model Context Protocol via MCPToolSet
- **Multi-Agent Systems**: Build complex workflows with multiple agents

### MCPToolSet

`MCPToolSet` is a special ADK tool that automatically discovers and makes available all tools from an MCP server. Instead of manually defining each tool, the agent can connect to any MCP-compliant server and use its tools!

### Architecture:

```
┌──────────────┐         ┌─────────────┐         ┌──────────────┐
│    User      │◄───────►│  ADK Agent  │◄───────►│  MCPToolSet  │
│  (Natural    │  Chat   │  (Gemini)   │  Tools  │              │
│   Language)  │         │             │         │              │
└──────────────┘         └─────────────┘         └──────┬───────┘
                                                         │
                                                         │ HTTP/SSE
                                                         │
                                                  ┌──────▼───────┐
                                                  │  MCP Server  │
                                                  │  (Customer   │
                                                  │   Management)│
                                                  └──────────────┘
```

💡 **Learning Point**: ADK agents can use natural language to interact with structured tools, making it easy to build conversational interfaces for databases and APIs.

---

## 🔧 Installation and Setup

First, let's install Google ADK and its dependencies.

In [ ]:
# Install Google ADK
!pip install google-adk google-generativeai -q

print("✅ Google ADK installed successfully!")
print("📦 Installed: google-adk (Agent Development Kit), google-generativeai (Gemini API)")

---

## 🔑 Setup API Keys and MCP Server URL

We need two configuration values:
1. **Google API Key** - For accessing Gemini models
2. **MCP Server URL** - The ngrok URL from your running MCP server

### Getting Your Google API Key:

1. Go to [https://aistudio.google.com/app/apikey](https://aistudio.google.com/app/apikey)
2. Click "Create API Key"
3. Copy the generated key

### Adding Secrets to Colab:

1. Click the **🔑 (Secrets)** icon in the left sidebar
2. Add two secrets:
   - **Name**: `GOOGLE_API_KEY`, **Value**: Your Google API key
   - **Name**: `MCP_SERVER_URL`, **Value**: Your ngrok MCP endpoint (e.g., `https://abc123.ngrok.io/mcp`)
3. Enable **Notebook access** for both secrets

In [ ]:
from google.colab import userdata
import os
from termcolor import colored

# Get API key and MCP server URL from Colab secrets
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    print(colored("✅ Google API Key loaded", "green"))
except Exception as e:
    print(colored("❌ GOOGLE_API_KEY not found in secrets", "red"))
    print(colored("   Please add your Google API key to Colab secrets", "yellow"))
    print(colored("   Get one at: https://aistudio.google.com/app/apikey", "yellow"))
    raise

try:
    MCP_SERVER_URL = userdata.get('MCP_SERVER_URL')
    print(colored(f"✅ MCP Server URL loaded: {MCP_SERVER_URL}", "green"))
except Exception as e:
    print(colored("❌ MCP_SERVER_URL not found in secrets", "red"))
    print(colored("   Please add your MCP server URL to Colab secrets", "yellow"))
    print(colored("   Format: https://xxxx.ngrok.io/mcp", "yellow"))
    print(colored("   Run the mcp_customer_demo.ipynb notebook first to get this URL", "yellow"))
    raise

print()
print(colored("🎉 Configuration complete!", "cyan", attrs=["bold"]))

---

## 🤖 Create the ADK Agent with MCPToolSet

Now let's create our agent! The agent will:
1. Use the Gemini 2.5 Flash model
2. Connect to our MCP server via MCPToolSet
3. Automatically discover all 6 customer management tools
4. Be able to use these tools based on natural language requests

💡 **Learning Point**: MCPToolSet handles all the protocol communication with the MCP server. The agent doesn't need to know about JSON-RPC or SSE - it just sees the tools as functions it can call!

In [ ]:
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import MCPToolset, StreamableHTTPConnectionParams

print(colored("🔧 Creating ADK Agent with MCP tools...", "cyan"))

# Create the agent
customer_agent = LlmAgent(
    model="gemini-2.5-flash",
    name="customer_management_agent",
    description="An AI agent that helps manage customer data using MCP tools",
    tools=[
        MCPToolset(
            connection_params=StreamableHTTPConnectionParams(
                url=MCP_SERVER_URL
            )
        )
    ],
    system_instruction="""You are a helpful customer management assistant.
    
You have access to a customer database through MCP tools. Use these tools to help users:
- Get information about specific customers
- List all customers or filter by status
- Add new customers
- Update customer information
- Disable or activate customer accounts

Always provide clear, friendly responses. When performing operations, explain what you're doing.
Format customer information in a clear, readable way."""
)

print(colored("✅ Agent created successfully!", "green", attrs=["bold"]))
print()
print(colored("📋 Agent Details:", "cyan"))
print(f"   Name: {customer_agent.name}")
print(f"   Model: gemini-2.5-flash")
print(f"   Tools: MCPToolSet connected to {MCP_SERVER_URL}")
print()
print(colored("💡 The agent can now use all customer management tools!", "yellow"))

---

## 🧪 Test the Agent

Let's test our agent with various customer management scenarios. The agent will understand natural language requests and use the appropriate MCP tools to fulfill them.

### Helper Function for Agent Interaction

In [ ]:
def ask_agent(query: str, show_full_response: bool = False):
    """
    Send a query to the agent and display the response.
    
    Args:
        query: The question or request to send to the agent
        show_full_response: If True, show the raw response object
    """
    print(colored("="*70, "magenta"))
    print(colored(f"👤 USER: {query}", "cyan", attrs=["bold"]))
    print(colored("="*70, "magenta"))
    print()
    
    try:
        # Send the query to the agent
        response = customer_agent.generate_response(query)
        
        # Extract the text response
        if hasattr(response, 'text'):
            agent_response = response.text
        elif hasattr(response, 'content'):
            agent_response = response.content
        else:
            agent_response = str(response)
        
        print(colored("🤖 AGENT:", "green", attrs=["bold"]))
        print(agent_response)
        print()
        
        if show_full_response:
            print(colored("📊 Full Response Object:", "yellow"))
            print(response)
            print()
        
        return response
        
    except Exception as e:
        print(colored(f"❌ Error: {e}", "red"))
        print()
        return None

print(colored("✅ Helper function defined", "green"))

---

## Test 1: List All Customers

Let's start with a simple request to list all customers.

In [ ]:
ask_agent("Can you show me all the customers in the database?")

---

## Test 2: Get Specific Customer

Request information about a specific customer by ID.

In [ ]:
ask_agent("What information do you have about customer ID 1?")

---

## Test 3: Filter by Status

Request only active customers.

In [ ]:
ask_agent("Show me only the active customers")

---

## Test 4: Add New Customer

Test adding a new customer with natural language.

In [ ]:
ask_agent("Please add a new customer named 'John Wick' with email 'john.wick@continental.com' and phone '+1-555-KILLER'")

---

## Test 5: Update Customer Information

Update an existing customer's information.

In [ ]:
ask_agent("Update customer ID 2's phone number to '+1-555-NEW-PHONE'")

---

## Test 6: Disable Customer Account

Test disabling a customer account.

In [ ]:
ask_agent("Disable customer ID 5's account")

---

## Test 7: Activate Customer Account

Reactivate a disabled customer.

In [ ]:
ask_agent("Please reactivate customer ID 4")

---

## Test 8: Complex Query

Test the agent's ability to understand and handle a multi-step request.

In [ ]:
ask_agent("Can you tell me how many active customers we have and who they are?")

---

## Test 9: Natural Language Search

Ask the agent to find specific information using natural language.

In [ ]:
ask_agent("Find me the customer named Alice Johnson and tell me their details")

---

## Test 10: Error Handling

Test how the agent handles requests for non-existent data.

In [ ]:
ask_agent("Show me customer ID 99999")

---

## 🎮 Interactive Chat

Now you can interact with the agent freely! Try your own queries.

In [ ]:
# Try your own query here!
ask_agent("List all disabled customers")

---

## 🌐 Deploy Agent with ADK Web + Ngrok

Now let's deploy the agent with a web interface that's publicly accessible!

**ADK Web** provides a chat UI for your agent. Combined with ngrok, anyone can access your agent from anywhere.

### What We'll Do:

1. Save the agent configuration to a file
2. Install pyngrok for tunneling
3. Start ADK web server
4. Create a public URL with ngrok
5. Share the URL for others to chat with your agent!

💡 **Learning Point**: `adk web` automatically creates a chat interface for your agent. No need to build a UI from scratch!

### Step 1: Save Agent Configuration

ADK Web needs the agent configuration in a Python file.

In [ ]:
# Create agent.py file for ADK Web
agent_code = f'''"""Customer Management Agent Configuration"""
import os
from google.adk.agents import LlmAgent
from google.adk.tools.mcp_tool import MCPToolset, StreamableHTTPConnectionParams

# Get configuration from environment
MCP_SERVER_URL = os.getenv("MCP_SERVER_URL", "{MCP_SERVER_URL}")

# Create the agent
root_agent = LlmAgent(
    model="gemini-2.5-flash",
    name="customer_management_agent",
    description="An AI agent that helps manage customer data using MCP tools",
    tools=[
        MCPToolset(
            connection_params=StreamableHTTPConnectionParams(
                url=MCP_SERVER_URL
            )
        )
    ],
    system_instruction="""You are a helpful customer management assistant.
    
You have access to a customer database through MCP tools. Use these tools to help users:
- Get information about specific customers
- List all customers or filter by status
- Add new customers
- Update customer information
- Disable or activate customer accounts

Always provide clear, friendly responses. When performing operations, explain what you\'re doing.
Format customer information in a clear, readable way."""
)
'''

# Write to file
with open('/content/agent.py', 'w') as f:
    f.write(agent_code)

print(colored("✅ Agent configuration saved to /content/agent.py", "green"))
print(colored("📄 This file defines the 'root_agent' for ADK Web", "cyan"))

### Step 2: Install pyngrok

We need pyngrok to create the public tunnel.

In [ ]:
!pip install pyngrok -q

print(colored("✅ pyngrok installed", "green"))

### Step 3: Start ADK Web with Ngrok

This cell will:
1. Start the ADK web server in the background
2. Create an ngrok tunnel
3. Display the public URL

**Important**: The server will keep running until you stop it or reset the Colab runtime.

**Note**: Make sure you have `NGROK_AUTHTOKEN` in your Colab secrets (from the MCP server notebook).

In [ ]:
import subprocess
import time
from pyngrok import ngrok
from google.colab import userdata
import signal
import os

# Get ngrok auth token
try:
    ngrok_token = userdata.get('NGROK_AUTHTOKEN')
    ngrok.set_auth_token(ngrok_token)
    print(colored("✅ Ngrok authenticated", "green"))
except Exception as e:
    print(colored("❌ NGROK_AUTHTOKEN not found in secrets", "red"))
    print(colored("   Please add it to test with public URL", "yellow"))
    print(colored("   (The server will still start locally)", "yellow"))
    ngrok_token = None

print()
print(colored("🚀 Starting ADK Web server...", "cyan", attrs=["bold"]))
print()

# Start ADK web server in background
process = subprocess.Popen(
    ['adk', 'web', '--agent_path', '/content/agent.py', '--port', '8000'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# Wait for server to start
time.sleep(5)

# Check if server is running
if process.poll() is None:
    print(colored("✅ ADK Web server is running!", "green", attrs=["bold"]))
    print(colored("📍 Local URL: http://localhost:8000", "cyan"))
    print()
    
    # Create ngrok tunnel if token is available
    if ngrok_token:
        try:
            public_url = ngrok.connect(8000)
            print(colored("🌐 Public URL:", "green", attrs=["bold"]))
            print(colored(f"   {public_url}", "green", attrs=["bold", "underline"]))
            print()
            print(colored("📱 Share this URL with anyone to chat with your agent!", "yellow"))
            print()
            print(colored("💡 The agent has access to:", "cyan"))
            print("   • get_customer - Retrieve customer by ID")
            print("   • list_customers - List all or filtered customers")
            print("   • add_customer - Create new customer")
            print("   • update_customer - Update customer info")
            print("   • disable_customer - Deactivate account")
            print("   • activate_customer - Reactivate account")
            print()
            print(colored("⚠️  Keep this cell running to keep the server active!", "yellow", attrs=["bold"]))
        except Exception as e:
            print(colored(f"⚠️  Could not create ngrok tunnel: {e}", "yellow"))
            print(colored("   Server is still accessible locally at http://localhost:8000", "cyan"))
else:
    print(colored("❌ Server failed to start", "red"))
    print("Error output:")
    print(process.stderr.read())

### Using the Web Interface

Once the server is running:

1. **Open the public URL** in your browser (or http://localhost:8000 in Colab)
2. **Start chatting** with the agent in natural language
3. **Try queries like**:
   - "Show me all customers"
   - "Add a new customer named Jane Doe with email jane@example.com"
   - "What's the status of customer ID 3?"
   - "List only active customers"

### Features of ADK Web:

- ✅ **Chat UI** - Clean, modern interface
- ✅ **Real-time** - Instant responses
- ✅ **Tool calls visible** - See when the agent uses MCP tools
- ✅ **Conversation history** - Maintains context
- ✅ **Mobile-friendly** - Works on any device

### Stopping the Server

To stop the server:
- **Runtime → Interrupt execution** in Colab menu
- Or **Runtime → Restart runtime** to fully reset

---

## 📝 Summary and Learning Points

### What We Built

Congratulations! You've successfully created an AI agent using Google ADK that:

1. **Connects to MCP Servers** - Used MCPToolSet to connect to our customer management MCP server
2. **Understands Natural Language** - Processes user requests in plain English
3. **Uses Tools Intelligently** - Automatically selects the right MCP tool for each task
4. **Provides Clear Responses** - Formats data in a user-friendly way

### Key Concepts Learned

#### 1. Google ADK Architecture
```python
LlmAgent(
    model="gemini-2.5-flash",
    name="agent_name",
    tools=[...],  # Tools the agent can use
    system_instruction="..."  # Agent's behavior
)
```

#### 2. MCPToolSet Integration
```python
MCPToolset(
    connection_params=StreamableHTTPConnectionParams(
        url="https://your-mcp-server/mcp"
    )
)
```

#### 3. Tool Discovery
- MCPToolSet automatically discovers all tools from the MCP server
- No need to manually define each tool
- Tools are made available to the agent with their schemas

#### 4. Natural Language to Tool Calls
- The agent understands user intent
- Selects appropriate tools
- Handles parameters and formats responses

### Advantages of This Approach

✅ **Separation of Concerns**: MCP server handles data, ADK agent handles conversation

✅ **Reusability**: Same MCP server can be used by multiple agents or clients

✅ **Maintainability**: Update tools on the server without changing agent code

✅ **Scalability**: Add more tools to the MCP server, agent discovers them automatically

✅ **User Experience**: Natural language interface is much easier than APIs

### Real-World Applications

This pattern can be extended to:
- **Customer Support Bots** - AI agents that help customers using backend systems
- **Internal Tools** - Conversational interfaces for employee databases
- **Sales Assistants** - AI that helps sales teams access CRM data
- **Data Analysis** - Natural language queries for business intelligence
- **Multi-Agent Systems** - Multiple agents working together with different MCP servers

### Next Steps

To extend this demo:
1. **Add Memory**: Use ADK's conversation history features
2. **Multi-Agent**: Create specialized agents for different tasks
3. **Add More MCP Servers**: Connect to multiple data sources
4. **Build UI**: Create a chat interface with Gradio or Streamlit
5. **Deploy**: Host the agent as a web service

### Resources

- **Google ADK Documentation**: [google.github.io/adk-docs](https://google.github.io/adk-docs/)
- **MCP Tools Guide**: [google.github.io/adk-docs/tools/mcp-tools](https://google.github.io/adk-docs/tools/mcp-tools/)
- **ADK Python Repository**: [github.com/google/adk-python](https://github.com/google/adk-python)
- **Gemini API**: [ai.google.dev](https://ai.google.dev)

---

## 💡 Final Thoughts

By combining Google ADK with MCP, you can:
- Build conversational interfaces for any API or database
- Create agents that work with existing systems via MCP
- Provide natural language access to structured data
- Build sophisticated multi-agent systems

The combination of ADK (for agent intelligence) and MCP (for tool standardization) creates a powerful platform for building AI applications!

**Happy Building!** 🚀